In [3]:
import csv
import numpy as np
import string

In [4]:
content_list = []
with open("trg.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        content_list.append(row)

In [5]:
content_list[1]

['1',
 'B',
 'the 4 202 353 bp genome of the alkaliphilic bacterium bacillus halodurans c-125 contains 4066 predicted protein coding sequences cdss 2141 527 of which have functional assignments 1182 29 of which are conserved cdss with unknown function and 743 18 3 of which have no match to any protein database among the total cdss 88 match sequences of proteins found only in bacillus subtilis and 667 are widely conserved in comparison with the proteins of various organisms including bsubtilis the b halodurans genome contains 112 transposase genes indicating that transposases have played an important evolutionary role in horizontal gene transfer and also in internal genetic rearrangement in the genome strain c-125 lacks some of the necessary genes for competence such as coms srfa and rapc supporting the fact that competence has not been demonstrated experimentally in c-125 there is no paralog of tupa encoding teichuronopeptide which contributes to alkaliphily in the c-125 genome and an 

In [6]:
#Extract class label
class_label = [x[1] for x in content_list[1:]]
#Extract content
content = [x[2].lower().split() for x in content_list[1:]]


In [7]:
def createVocabList(dataSet):  #create wordlist
    word_dict = {}
    for document in dataSet:
        for word in document:
            if word not in word_dict:
                word_dict[word] = 1
            else:
                word_dict[word] += 1
    return word_dict

## Create stopword list

In [8]:
import nltk
from nltk.corpus import stopwords


def removePunctuation(value):
    result = ''
    for a in value:
        if a not in string.punctuation:
            result += a
    return result



stop_word = []
for stopword in stopwords.words('english'):
    stopword = removePunctuation(stopword)
    stop_word.append(stopword)

## find out the most 1000 frequent words 

In [9]:
most_occurring_words_1000 = [i[0] 
                             for i in sorted(createVocabList(content).items(), key = lambda kv:(kv[1], kv[0]),reverse=True) 
                             if i[0] not in stop_word][0:1000]


## One-hot encoder

In [10]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1#multivariate NB
    return returnVec

In [11]:
def listOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1#multinomial NB
    return returnVec

In [12]:
train_matrix = []
for i in content:
    train_matrix.append(setOfWords2Vec(most_occurring_words_1000,i))

In [13]:
train_matrix_extension = []
for i in content:
    train_matrix_extension.append(listOfWords2Vec(most_occurring_words_1000,i))

In [14]:
train_matrix = np.array(train_matrix)
train_matrix

array([[1, 0, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [15]:
train_matrix_extension = np.array(train_matrix_extension)
train_matrix_extension

array([[2, 0, 1, ..., 0, 0, 0],
       [4, 1, 7, ..., 0, 0, 0],
       [6, 3, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [2, 3, 0, ..., 0, 0, 0],
       [4, 1, 0, ..., 0, 0, 0]])

In [16]:
class_label = np.array(class_label)
class_label

array(['B', 'A', 'E', ..., 'E', 'E', 'B'], dtype='<U1')

In [19]:
combined_train_matrix = np.column_stack((train_matrix,class_label))
combined_train_matrix

array([['1', '0', '1', ..., '0', '0', 'B'],
       ['1', '1', '1', ..., '0', '0', 'A'],
       ['1', '1', '1', ..., '0', '0', 'E'],
       ...,
       ['0', '1', '0', ..., '0', '0', 'E'],
       ['1', '1', '0', ..., '0', '0', 'E'],
       ['1', '1', '0', ..., '0', '0', 'B']], dtype='<U21')

In [20]:
combined_train_matrix_extension = np.column_stack((train_matrix_extension,class_label))
combined_train_matrix_extension

array([['2', '0', '1', ..., '0', '0', 'B'],
       ['4', '1', '7', ..., '0', '0', 'A'],
       ['6', '3', '1', ..., '0', '0', 'E'],
       ...,
       ['0', '1', '0', ..., '0', '0', 'E'],
       ['2', '3', '0', ..., '0', '0', 'E'],
       ['4', '1', '0', ..., '0', '0', 'B']], dtype='<U21')

## Train NB model

In [25]:
def trainNB(train_matrix,class_label):
    
    train_matrix = train_matrix.astype(float)
    num_of_Words = len(train_matrix[0])#1000

    #apply laplace smoothing
    pA_Num = np.ones(num_of_Words)
    pB_Num = np.ones(num_of_Words)
    pE_Num = np.ones(num_of_Words)
    pV_Num = np.ones(num_of_Words)

    pA_Denom = 1000
    pB_Denom = 1000
    pE_Denom = 1000
    pV_Denom = 1000

    for i in range(len(train_matrix)):
        if class_label[i] == "A":
            pA_Num += train_matrix[i]
            pA_Denom += sum(train_matrix[i])
        elif class_label[i] == "B":
            pB_Num += train_matrix[i]
            pB_Denom += sum(train_matrix[i])
        elif class_label[i] == "E":
            pE_Num += train_matrix[i]
            pE_Denom += sum(train_matrix[i])
        elif class_label[i] == "V":
            pV_Num += train_matrix[i]
            pV_Denom += sum(train_matrix[i])
    
    #logrithmically
    pA_Vect = np.log(pA_Num / pA_Denom) #p(data|A) vector
    pB_Vect = np.log(pB_Num / pB_Denom)
    pE_Vect = np.log(pE_Num / pE_Denom)
    pV_Vect = np.log(pV_Num / pV_Denom)
    
    return pA_Vect,pB_Vect,pE_Vect,pV_Vect

In [26]:
def trainNB_extension(train_matrix,class_label,boolean_matrix):
    
    train_matrix = train_matrix.astype(float)
    num_of_Words = len(train_matrix[0]) #feature number
    instance_length = len(train_matrix)

    #apply laplace smoothing
    pA_Num = np.ones(num_of_Words)
    pB_Num = np.ones(num_of_Words)
    pE_Num = np.ones(num_of_Words)
    pV_Num = np.ones(num_of_Words)

    pA_Denom = num_of_Words
    pB_Denom = num_of_Words
    pE_Denom = num_of_Words
    pV_Denom = num_of_Words
    
    #inverse document frequency
    count_list = np.zeros(num_of_Words)
    for i in range(num_of_Words):
        #count the total frequency of each word
        count = np.sum(boolean_matrix[:,i])
        count_list[i] = count
        
    temp_list = []
    for i in range(num_of_Words):
        #calculate the weight
        column_result = np.log(len(train_matrix)/count_list[i]) * train_matrix[:,i]        
        temp_list.append(column_result)
        
    train_matrix = np.array(temp_list).T

    for i in range(len(train_matrix)):
        if class_label[i] == "A":
            pA_Num += train_matrix[i]
            pA_Denom += sum(train_matrix[i])
        elif class_label[i] == "B":
            pB_Num += train_matrix[i]
            pB_Denom += sum(train_matrix[i])
        elif class_label[i] == "E":
            pE_Num += train_matrix[i]
            pE_Denom += sum(train_matrix[i])
        elif class_label[i] == "V":
            pV_Num += train_matrix[i]
            pV_Denom += sum(train_matrix[i])
    
    #logrithmically
    pA_Vect = np.log(pA_Num / pA_Denom) #p(data|A) vector
    pB_Vect = np.log(pB_Num / pB_Denom)
    pE_Vect = np.log(pE_Num / pE_Denom)
    pV_Vect = np.log(pV_Num / pV_Denom)
    
    return pA_Vect,pB_Vect,pE_Vect,pV_Vect

## Predict test label

In [27]:
def generate_label_result(test_set,pa,pb,pe,pv,prior_prob_list):
    test_set = test_set.astype(float)
    result_pA = np.sum(test_set*pa,axis=1)+np.log(prior_prob_list[0])
    result_pB = np.sum(test_set*pb,axis=1)+np.log(prior_prob_list[1])
    result_pE = np.sum(test_set*pe,axis=1)+np.log(prior_prob_list[2])
    result_pV = np.sum(test_set*pv,axis=1)+np.log(prior_prob_list[3])
    test_label = []
    for i in range(len(result_pA)):
        max_value = max(result_pA[i],result_pB[i],result_pE[i],result_pV[i])
        if max_value == result_pA[i]:
            test_label.append("A")
        elif max_value == result_pB[i]:
            test_label.append("B")
        elif max_value == result_pE[i]:
            test_label.append("E")
        else:
            test_label.append("V")
    return test_label

## Cross-validation

In [28]:
#split data

def cross_validation(K,dataset):
    np.random.seed(0)
    shuffled_dataset = np.random.permutation(dataset)
    testing = []
    
    training = []
    each_proportion = int(len(train_matrix)/K)
    for i in range(K):
        testing.append(shuffled_dataset[each_proportion*i:each_proportion*(i+1)])
        temp = []
        for j in range(K):
            
            if i != j:
                temp.append(shuffled_dataset[each_proportion*j:each_proportion*(j+1)])
                
        training.append(np.concatenate(temp))
        
    return testing,training



In [29]:
testing_set,training_set = cross_validation(10,combined_train_matrix)

In [30]:
testing_set_extension,training_set_extension = cross_validation(10,combined_train_matrix_extension)

In [31]:
def cross_val_score(K,training_set,testing_set,training_model):
    print("Training model...")
    class_label_list = ["A","B","E","V"]
    prior_prob_list = []
    score_list = []
    for k in range(K):
        train_k = training_set[k]
        if signal == True:
            pa,pb,pe,pv = training_model(train_k[:,:-1],train_k[:,-1],boolean_matrix[k][:,:-1].astype(float))
        else:
            pa,pb,pe,pv = training_model(train_k[:,:-1],train_k[:,-1])
        for i in class_label_list:
            p= len([a for a in train_k[:,-1] if a == i])/len(train_k[:,-1])
            prior_prob_list.append(p)
            
        test_label = generate_label_result(testing_set[k][:,:-1],pa,pb,pe,pv,prior_prob_list)
        
        result1 = 0
        for j in range(len(test_label)):
            result1 += 1 if (list(test_label)[j] == list(testing_set[k][:,-1])[j]) else 0
        score = float(result1/len(test_label))
        score_list.append(score)
    
    
    return f"""The mean accuarcy is of {K}-fold validation is {np.mean(score_list)}"""
        
            

## Standard NB

In [32]:
signal = False
Mean_score = cross_val_score(10,training_set,testing_set,trainNB)

Training model...


In [33]:
Mean_score

'The mean accuarcy is of 10-fold validation is 0.91325'

## Extension NB

In [34]:
signal = True
boolean_matrix = training_set
Mean_score_extension = cross_val_score(10,training_set_extension,testing_set,trainNB_extension)

Training model...


In [35]:
Mean_score_extension

'The mean accuarcy is of 10-fold validation is 0.92225'

## Prediction on test file

In [36]:
class_label_list = ["A","B","E","V"]
prior_prob = []
for i in class_label_list:
    p= len([a for a in combined_train_matrix[:,-1] if a == i])/len(combined_train_matrix[:,-1])
    prior_prob.append(p)

In [37]:
prior_prob

[0.032, 0.4005, 0.536, 0.0315]

In [38]:
test_content = []
with open("tst.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        test_content.append(row)

In [39]:
content = [x[1].lower().split() for x in test_content[1:]]
test_matrix = []
for i in content:
    test_matrix.append(setOfWords2Vec(most_occurring_words_1000,i))

In [40]:
test_matrix = np.array(test_matrix)
test_matrix

array([[1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [41]:
pa,pb,pe,pv = trainNB_extension(combined_train_matrix_extension[:,:-1],combined_train_matrix[:,-1],combined_train_matrix[:,:-1].astype(float))
predict = generate_label_result(test_matrix,pa,pb,pe,pv,prior_prob)


In [211]:
with open('wxu520.csv', 'w', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    writer.writerow(["id","class"])
    for i,label in enumerate(predict):
        writer.writerow([i+1,label])